In [2]:
import numpy as np
%load_ext Cython

In [3]:
"""
Most of the BJ stuff should follow general game conventions. 
We initially model more intricate details (burning, shuffling protocol) based on local casino.
https://www.crownmelbourne.com.au/getsydmedia/99be12f2-c48b-44dc-b6ba-ab5849f6f723/crown-melbourne-gaming-blackjack-rules_1.pdf?ext=.pdf
https://en.wikipedia.org/wiki/Glossary_of_blackjack_terms
"""

# configurations
NDECKS: int = 8
# NPLAYERS: int = 4 # affects how many cards you may be able to count
DECK_PENETRATION: int = 2 # typical number of decks dealt out before reshuffle

In [4]:
"""
We keep our representation of the deck and cards as characters, rather than grouping on BJ value
Importantly, this means our framework remains flexible enough to add in weird house rules such as perfect pairs/flushes
"""
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A']) # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'])
deck = np.array([np.char.add(cards, suit) for suit in suits]).ravel()
shoe = np.repeat(deck, NDECKS)

# We use bytestrings to enable Cython memview of C char[2] dtypes
shoe = shoe.astype('S2')

numcardsdeal = len(deck) * DECK_PENETRATION

In [5]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(shoe)

8.05 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
# bad idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards.astype('S1'), values))
%timeit np.array([dict_map[x[:1]] for x in shoe]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(shoe)

66.7 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
# may change to use numpy to read-in csv, to avoid pandas dependency
import pandas as pd
strat = pd.read_csv(r"StrategyInput\BasicNoDeviations-4to8Decks-HitSoft17.csv", header=None)

"""
Read in the individual strategy tables, and set top left corner position to 0.
"""
hard = strat.iloc[0:18, :].copy().reset_index(drop=True)
hard[0][0] = '0'
hard = hard.values.astype(str)
hrd = hard[1:,1:].astype('S1')

soft = strat.iloc[18:27, :].copy().reset_index(drop=True)
soft[0][0] = '0'
soft = soft.values.astype(str)
sft = soft[1:,1:].astype('S1')

splits = strat.iloc[27:38, :].copy().reset_index(drop=True)
splits[0][0] = '0'
splits = splits.values.astype(str)
splts = splits[1:,1:].astype('S1')

In [8]:
count = pd.read_csv(r"StrategyInput\HiLoCount.csv", header=None)
cnt = count.iloc[:,1].values.astype(float)

In [37]:
%%cython --cplus
# do not know if this global directive is actually needed - maybe later when we break into files
# # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
# cython: c_string_type=str, c_string_encoding=ascii

import numpy as np
cimport cython
from libcpp.vector cimport vector

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# C uses ascii integer representation for characters
# Numeric characters in our shoe will have an ascii value less than ascii of 'a'
# English characters will have an ascii value greater than ascii of 'a'
# https://www.cs.cmu.edu/~pattis/15-1XX/common/handouts/ascii.html
cdef int ASCII_A = 65 

# Python dicts actually implemented quite fast
# If this is too slow, can consider using unordered_map from C++ STL. 
# https://stackoverflow.com/questions/29268914/python-dictionaries-vs-c-stdunordered-map-cython-vs-cythonized-python-dict
# https://stackoverflow.com/questions/32266444/using-a-dictionary-in-cython-especially-inside-nogil
cdef dict dealer_idx = {
    b'2': 0,
    b'3': 1,
    b'4': 2,
    b'5': 3,
    b'6': 4,
    b'7': 5,
    b'8': 6,
    b'9': 7,
    b'T': 8,
    b'J': 8,
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}

# function decorators to turn off various things, in the interest of speed
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(
    card_dtype_struct[:] shoe not None, 
    int numcardsdeal, 
    char[:,:] hrd not None, 
    char[:,:] sft not None, 
    char[:,:] splt not None, 
    double[:] cnt not None):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 

    Inputs: (REFER TO TEMPLATE FILES IN STRATEGY INPUT)
        - 'shoe': numpy array (buffer) of char[2] byte strings
        - 'numcardsdeal': int, simulates the depth of the cut card
        - 'hrd': numpy array (buffer) of char bytes, in the input format dictating BJ action on player hard hand
        - 'sft': numpy array (buffer) of char bytes, in the input format dictating BJ action on player soft hand
        - 'splt': numpy array (buffer) of char bytes, in the input format dictating BJ action on player paired hand
        - 'cnt': numpy array (buffer) of double (Python float6), in the input format, with count values
    """
    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef double count = 0 # card count

    # Use vector from cpp STL to represent player and dealer hands
    cdef vector[char] phand = vector[char](10)

    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
    # while dealt < numcardsdeal:
        
    



    print(bool(shoe[0].face == b'5'))

    # type cast into python bytes, so works as dict key
    # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
    print(dealer_idx.get(<bytes>(shoe[0].face)))




In [38]:
play_shoe(shoe, numcardsdeal, hrd, sft, splts, cnt)

True
3


In [13]:
shoe[:50]

array([b'5C', b'8C', b'7C', b'9C', b'9D', b'TS', b'6H', b'QH', b'QC',
       b'AS', b'TS', b'TS', b'3S', b'2D', b'5H', b'8D', b'5S', b'TH',
       b'6S', b'8H', b'TD', b'QD', b'KC', b'4D', b'TD', b'6S', b'AD',
       b'KD', b'4D', b'3D', b'KC', b'2S', b'7C', b'AD', b'AD', b'3S',
       b'QS', b'6C', b'7D', b'QC', b'4S', b'4C', b'4H', b'JH', b'AS',
       b'QS', b'3H', b'AC', b'TH', b'AC'], dtype='|S2')

In [35]:
hard

array([['0', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'A'],
       ['4', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['5', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['6', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['7', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['8', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['9', 'H', 'D', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['10', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'H', 'H'],
       ['11', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'],
       ['12', 'H', 'H', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['13', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['14', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['15', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'R', 'R'],
       ['16', 'S', 'S', 'S', 'S', 'S', 'H', 'H', 'R', 'R', 'R'],
       ['17', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'R'],
       ['18', 'S', 'S', 'S', 'S'